# Lab 6: AI Agents & Tool Calling

**Duration**: 120 minutes  
**Level**: Advanced

## Overview

In this lab, you'll learn how to build AI agents that can interact with external tools and systems. We'll explore:

- Function/tool calling fundamentals
- Building calculator agents with OpenAI and Claude
- Multi-tool assistants
- Conditional workflows and routing logic
- Error handling and resilient agents
- Multi-agent systems (AgentHub)

By the end of this lab, you'll have built a complete multi-agent platform capable of routing queries to specialized agents.

## Setup

First, let's verify our environment and install required packages.

In [ ]:
# Verify Python version
import sys
print(f"Python version: {sys.version}")

# Install required packages
!pip install openai anthropic python-dotenv

In [ ]:
# Import required libraries
import os
import json
from datetime import datetime
from typing import List, Dict, Any, Callable
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API keys
openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")

print(f"OpenAI API Key present: {bool(openai_key)}")
print(f"Anthropic API Key present: {bool(anthropic_key)}")

## Understanding Tool Calling

**Tool calling** (also called function calling) allows LLMs to:
1. Recognize when they need external information or capabilities
2. Request specific tool usage with parameters
3. Receive tool results and incorporate them into responses

### The Tool Calling Flow:

```
User Query → LLM decides tool needed → LLM returns tool call request
    ↓
Your code executes tool → Returns result to LLM
    ↓
LLM generates final response using tool result
```

### Key Concepts:

- **Tool Definition**: JSON schema describing the tool's name, description, and parameters
- **Tool Call**: LLM's request to use a tool with specific arguments
- **Tool Result**: The output from executing the tool
- **Tool Integration**: Feeding results back to the LLM for final response generation

## Exercise 1: Basic Calculator Agent

Let's build a simple calculator agent that can perform arithmetic operations.

### Part A: OpenAI Calculator Agent

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_key)

# Define calculator tool
calculator_tool = {
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Performs basic arithmetic operations: add, subtract, multiply, divide",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "The arithmetic operation to perform"
                },
                "a": {
                    "type": "number",
                    "description": "The first number"
                },
                "b": {
                    "type": "number",
                    "description": "The second number"
                }
            },
            "required": ["operation", "a", "b"]
        }
    }
}

# Implement calculator function
def calculator(operation: str, a: float, b: float) -> float:
    """Execute calculator operations."""
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Cannot divide by zero")
        return a / b
    else:
        raise ValueError(f"Unknown operation: {operation}")

print("Calculator tool defined successfully!")

In [ ]:
# Simple calculator agent with OpenAI
def calculator_agent_openai(user_message: str) -> str:
    """Calculator agent using OpenAI function calling."""
    
    # Step 1: Send message to LLM with tool definition
    messages = [{"role": "user", "content": user_message}]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=[calculator_tool],
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    
    # Step 2: Check if LLM wants to call a tool
    if response_message.tool_calls:
        # Extract tool call details
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        print(f"Tool called: {function_name}")
        print(f"Arguments: {function_args}")
        
        # Step 3: Execute the tool
        result = calculator(**function_args)
        print(f"Tool result: {result}")
        
        # Step 4: Send tool result back to LLM
        messages.append(response_message)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(result)
        })
        
        # Step 5: Get final response from LLM
        final_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        
        return final_response.choices[0].message.content
    else:
        # No tool call needed
        return response_message.content

# Test the agent
print("Testing OpenAI Calculator Agent:")
print("="*50)
result = calculator_agent_openai("What is 45 multiplied by 12?")
print(f"\nFinal response: {result}")

### Part B: Claude Calculator Agent

Now let's implement the same calculator using Claude's tool calling API.

In [ ]:
from anthropic import Anthropic

anthropic_client = Anthropic(api_key=anthropic_key)

# Define calculator tool for Claude
calculator_tool_claude = {
    "name": "calculator",
    "description": "Performs basic arithmetic operations: add, subtract, multiply, divide",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The arithmetic operation to perform"
            },
            "a": {
                "type": "number",
                "description": "The first number"
            },
            "b": {
                "type": "number",
                "description": "The second number"
            }
        },
        "required": ["operation", "a", "b"]
    }
}

print("Claude calculator tool defined successfully!")

In [ ]:
# Calculator agent with Claude
def calculator_agent_claude(user_message: str) -> str:
    """Calculator agent using Claude tool calling."""
    
    # Step 1: Send message to Claude with tool definition
    response = anthropic_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        tools=[calculator_tool_claude],
        messages=[{"role": "user", "content": user_message}]
    )
    
    # Step 2: Check if Claude wants to use a tool
    if response.stop_reason == "tool_use":
        # Find the tool use block
        tool_use = next(block for block in response.content if block.type == "tool_use")
        
        function_name = tool_use.name
        function_args = tool_use.input
        
        print(f"Tool called: {function_name}")
        print(f"Arguments: {function_args}")
        
        # Step 3: Execute the tool
        result = calculator(**function_args)
        print(f"Tool result: {result}")
        
        # Step 4: Send tool result back to Claude
        final_response = anthropic_client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1024,
            tools=[calculator_tool_claude],
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": response.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": str(result)
                        }
                    ]
                }
            ]
        )
        
        # Step 5: Extract final text response
        text_block = next(block for block in final_response.content if hasattr(block, "text"))
        return text_block.text
    else:
        # No tool call needed
        text_block = next(block for block in response.content if hasattr(block, "text"))
        return text_block.text

# Test the agent
print("Testing Claude Calculator Agent:")
print("="*50)
result = calculator_agent_claude("What is 45 multiplied by 12?")
print(f"\nFinal response: {result}")

### 🎯 Checkpoint 1

You should now understand:
- How to define tools/functions for LLMs
- The tool calling flow: request → execute → return result → final response
- Differences between OpenAI and Claude tool calling APIs

**Key Differences:**
- OpenAI uses `tools` array with `type: "function"` wrapper
- Claude uses `tools` array with direct tool definitions and `input_schema`
- OpenAI returns `tool_calls` in message; Claude uses `stop_reason: "tool_use"`
- Tool result handling differs slightly in message structure

## Exercise 2: Multi-Tool Assistant

Let's build an assistant with multiple tools: calculator, datetime, knowledge search, and notifications.

### Step 1: Define Multiple Tools

In [ ]:
# Tool implementations
def get_current_datetime() -> str:
    """Returns the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def search_knowledge_base(query: str) -> str:
    """Simulates searching a knowledge base."""
    # Simulated knowledge base
    kb = {
        "business hours": "Our business hours are Monday-Friday, 9 AM to 5 PM EST.",
        "return policy": "Items can be returned within 30 days with receipt.",
        "shipping": "We offer free shipping on orders over $50.",
        "support": "Contact support at support@example.com or call 1-800-HELP."
    }
    
    query_lower = query.lower()
    for key, value in kb.items():
        if key in query_lower:
            return value
    
    return "No relevant information found in knowledge base."

def send_notification(recipient: str, message: str, priority: str = "normal") -> str:
    """Simulates sending a notification."""
    return f"Notification sent to {recipient} (priority: {priority}): {message}"

print("All tool functions defined!")

In [ ]:
# Define tools for OpenAI
multi_tools_openai = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Performs basic arithmetic operations",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
                    "a": {"type": "number"},
                    "b": {"type": "number"}
                },
                "required": ["operation", "a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime",
            "description": "Returns the current date and time",
            "parameters": {"type": "object", "properties": {}}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "Searches the knowledge base for information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query"}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "send_notification",
            "description": "Sends a notification to a recipient",
            "parameters": {
                "type": "object",
                "properties": {
                    "recipient": {"type": "string", "description": "Email or name of recipient"},
                    "message": {"type": "string", "description": "The notification message"},
                    "priority": {"type": "string", "enum": ["low", "normal", "high"], "description": "Priority level"}
                },
                "required": ["recipient", "message"]
            }
        }
    }
]

print(f"Defined {len(multi_tools_openai)} tools for OpenAI")

### Step 2: Build Multi-Tool Agent

In [ ]:
# Tool registry for easy execution
TOOL_REGISTRY = {
    "calculator": calculator,
    "get_current_datetime": get_current_datetime,
    "search_knowledge_base": search_knowledge_base,
    "send_notification": send_notification
}

def multi_tool_agent(user_message: str, verbose: bool = True) -> str:
    """Agent that can use multiple tools."""
    
    messages = [{"role": "user", "content": user_message}]
    
    # Allow for multiple rounds of tool calls
    max_iterations = 5
    iteration = 0
    
    while iteration < max_iterations:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=multi_tools_openai,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        
        if not response_message.tool_calls:
            # No more tool calls, return final response
            return response_message.content
        
        # Process all tool calls in this response
        messages.append(response_message)
        
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if verbose:
                print(f"\n[Tool Call] {function_name}")
                print(f"[Arguments] {function_args}")
            
            # Execute tool
            tool_function = TOOL_REGISTRY.get(function_name)
            if tool_function:
                result = tool_function(**function_args)
                if verbose:
                    print(f"[Result] {result}")
            else:
                result = f"Error: Unknown tool {function_name}"
            
            # Add tool result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result)
            })
        
        iteration += 1
    
    return "Max iterations reached. Please try again."

print("Multi-tool agent ready!")

In [ ]:
# Test multi-tool agent
print("Test 1: Calculator query")
print("="*50)
response = multi_tool_agent("What is 234 divided by 13?")
print(f"\n[Final Response]\n{response}")

print("\n\nTest 2: Datetime query")
print("="*50)
response = multi_tool_agent("What time is it right now?")
print(f"\n[Final Response]\n{response}")

print("\n\nTest 3: Knowledge base query")
print("="*50)
response = multi_tool_agent("What are your business hours?")
print(f"\n[Final Response]\n{response}")

print("\n\nTest 4: Multi-step query")
print("="*50)
response = multi_tool_agent("Calculate 100 + 50, then notify admin@example.com about the result with high priority.")
print(f"\n[Final Response]\n{response}")

### 🎯 Checkpoint 2

You should now understand:
- How to define multiple tools for an agent
- Tool registry pattern for organizing tool functions
- Handling multiple tool calls in a single response
- Iterative tool calling for multi-step tasks

**Key Insights:**
- Agents can chain multiple tool calls together
- Tool results feed back into context for next decision
- Important to limit iterations to prevent infinite loops

## Exercise 3: Conditional Workflow Agent

Build an agent that routes queries based on conditions like business hours, urgency, or topic.

### Step 1: Define Workflow Tools

In [ ]:
def check_business_hours() -> Dict[str, Any]:
    """Check if current time is within business hours."""
    now = datetime.now()
    is_weekday = now.weekday() < 5  # Monday = 0, Sunday = 6
    is_business_hours = 9 <= now.hour < 17
    
    return {
        "is_open": is_weekday and is_business_hours,
        "current_time": now.strftime("%Y-%m-%d %H:%M:%S"),
        "day_of_week": now.strftime("%A")
    }

def route_to_email(email: str, subject: str, body: str) -> str:
    """Simulate routing query to email."""
    return f"Email sent to {email}\nSubject: {subject}\nBody: {body}"

def create_ticket(title: str, description: str, priority: str) -> Dict[str, Any]:
    """Simulate creating a support ticket."""
    import random
    ticket_id = f"TKT-{random.randint(10000, 99999)}"
    return {
        "ticket_id": ticket_id,
        "title": title,
        "description": description,
        "priority": priority,
        "status": "open",
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

print("Workflow tools defined!")

In [ ]:
# Define workflow tools for OpenAI
workflow_tools = [
    {
        "type": "function",
        "function": {
            "name": "check_business_hours",
            "description": "Check if current time is within business hours (Mon-Fri, 9 AM - 5 PM)",
            "parameters": {"type": "object", "properties": {}}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "route_to_email",
            "description": "Route the query to email support",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string", "description": "Recipient email address"},
                    "subject": {"type": "string", "description": "Email subject"},
                    "body": {"type": "string", "description": "Email body content"}
                },
                "required": ["email", "subject", "body"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_ticket",
            "description": "Create a support ticket",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string", "description": "Ticket title"},
                    "description": {"type": "string", "description": "Detailed description"},
                    "priority": {"type": "string", "enum": ["low", "medium", "high", "urgent"]}
                },
                "required": ["title", "description", "priority"]
            }
        }
    }
]

# Update tool registry
WORKFLOW_REGISTRY = {
    "check_business_hours": check_business_hours,
    "route_to_email": route_to_email,
    "create_ticket": create_ticket
}

print("Workflow tools configured!")

### Step 2: Build Conditional Workflow Agent

In [ ]:
def workflow_agent(user_message: str, verbose: bool = True) -> str:
    """Agent with conditional workflow logic."""
    
    system_prompt = """You are a customer support routing assistant. 
    
Your job is to:
1. Check business hours first
2. If outside business hours, create a ticket for the next day
3. If during business hours and urgent, route to email immediately
4. Otherwise, create a ticket with appropriate priority

Always be helpful and explain what you're doing."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    max_iterations = 5
    iteration = 0
    
    while iteration < max_iterations:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=workflow_tools,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        
        if not response_message.tool_calls:
            return response_message.content
        
        messages.append(response_message)
        
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if verbose:
                print(f"\n[Workflow Step] {function_name}")
                print(f"[Arguments] {function_args}")
            
            tool_function = WORKFLOW_REGISTRY.get(function_name)
            if tool_function:
                result = tool_function(**function_args)
                if verbose:
                    print(f"[Result] {result}")
            else:
                result = f"Error: Unknown tool {function_name}"
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result) if isinstance(result, dict) else str(result)
            })
        
        iteration += 1
    
    return "Max iterations reached."

print("Workflow agent ready!")

In [ ]:
# Test workflow agent
print("Test: Urgent issue during business hours")
print("="*60)
response = workflow_agent(
    "URGENT: Our payment system is down and we can't process orders! This is costing us money every minute."
)
print(f"\n[Final Response]\n{response}")

print("\n\nTest: Normal priority issue")
print("="*60)
response = workflow_agent(
    "I have a question about updating my billing address. Not urgent, but would like help when available."
)
print(f"\n[Final Response]\n{response}")

### 🎯 Checkpoint 3

You should now understand:
- Building conditional workflows with tool calling
- Using system prompts to guide agent decision-making
- Routing logic based on context (time, urgency, etc.)
- Multi-step workflows with different outcomes

**Key Patterns:**
- Check conditions first (business hours, availability)
- Route based on urgency and context
- Provide clear feedback to users about routing decisions

## Exercise 4: Resilient Agent with Error Handling

Build a production-ready agent with comprehensive error handling.

### Step 1: Define Error-Prone Tools

In [ ]:
class ToolExecutionError(Exception):
    """Custom exception for tool execution errors."""
    pass

def risky_calculator(operation: str, a: float, b: float) -> float:
    """Calculator that validates inputs and handles errors."""
    valid_operations = ["add", "subtract", "multiply", "divide"]
    
    if operation not in valid_operations:
        raise ToolExecutionError(f"Invalid operation '{operation}'. Must be one of {valid_operations}")
    
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise ToolExecutionError("Both operands must be numbers")
    
    if operation == "divide":
        if b == 0:
            raise ToolExecutionError("Division by zero is not allowed")
        return a / b
    elif operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b

def fetch_external_data(source: str) -> str:
    """Simulates fetching from external API (may fail)."""
    import random
    
    # Simulate random failures
    if random.random() < 0.3:  # 30% failure rate
        raise ToolExecutionError(f"Failed to fetch data from {source}: Connection timeout")
    
    return f"Data successfully retrieved from {source}"

print("Error-prone tools defined!")

### Step 2: Build Resilient Agent

In [ ]:
class ResilientAgent:
    """Production-ready agent with comprehensive error handling."""
    
    def __init__(self, max_retries: int = 3, max_iterations: int = 5):
        self.max_retries = max_retries
        self.max_iterations = max_iterations
        
        # Tool definitions
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "risky_calculator",
                    "description": "Performs arithmetic with validation",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
                            "a": {"type": "number"},
                            "b": {"type": "number"}
                        },
                        "required": ["operation", "a", "b"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "fetch_external_data",
                    "description": "Fetches data from external source (may fail)",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "source": {"type": "string", "description": "Data source name"}
                        },
                        "required": ["source"]
                    }
                }
            }
        ]
        
        self.tool_registry = {
            "risky_calculator": risky_calculator,
            "fetch_external_data": fetch_external_data
        }
    
    def execute_tool_with_retry(self, tool_name: str, tool_args: dict):
        """Execute tool with retry logic.
        
        Returns:
            Tuple[bool, Any]: (success, result)
        """
        tool_function = self.tool_registry.get(tool_name)
        if not tool_function:
            return False, f"Unknown tool: {tool_name}"
        
        for attempt in range(self.max_retries):
            try:
                result = tool_function(**tool_args)
                return True, result
            except ToolExecutionError as e:
                # Don't retry on validation errors
                return False, f"Tool error: {str(e)}"
            except Exception as e:
                # Retry on unexpected errors
                if attempt < self.max_retries - 1:
                    print(f"[Retry {attempt + 1}/{self.max_retries}] {tool_name} failed: {e}")
                    continue
                else:
                    return False, f"Tool failed after {self.max_retries} attempts: {str(e)}"
        
        return False, "Unexpected error in retry logic"
    
    def chat(self, user_message: str, verbose: bool = True) -> str:
        """Process user message with error handling."""
        messages = [{"role": "user", "content": user_message}]
        
        for iteration in range(self.max_iterations):
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages,
                    tools=self.tools,
                    tool_choice="auto"
                )
            except Exception as e:
                return f"LLM API error: {str(e)}"
            
            response_message = response.choices[0].message
            
            if not response_message.tool_calls:
                return response_message.content
            
            messages.append(response_message)
            
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                
                try:
                    function_args = json.loads(tool_call.function.arguments)
                except json.JSONDecodeError as e:
                    error_msg = f"Invalid tool arguments JSON: {str(e)}"
                    if verbose:
                        print(f"[Error] {error_msg}")
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": error_msg
                    })
                    continue
                
                if verbose:
                    print(f"\n[Tool Call] {function_name}")
                    print(f"[Arguments] {function_args}")
                
                success, result = self.execute_tool_with_retry(function_name, function_args)
                
                if verbose:
                    status = "Success" if success else "Failed"
                    print(f"[{status}] {result}")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
        
        return "Max iterations reached. Please try a simpler query."

print("ResilientAgent class defined!")

In [ ]:
# Test resilient agent
agent = ResilientAgent(max_retries=3)

print("Test 1: Valid calculator request")
print("="*60)
response = agent.chat("What is 100 divided by 4?")
print(f"\n[Final Response]\n{response}")

print("\n\nTest 2: Division by zero (should handle error)")
print("="*60)
response = agent.chat("Calculate 50 divided by 0")
print(f"\n[Final Response]\n{response}")

print("\n\nTest 3: External data fetch (may retry on failure)")
print("="*60)
response = agent.chat("Fetch data from the analytics database")
print(f"\n[Final Response]\n{response}")

### 🎯 Checkpoint 4

You should now understand:
- Comprehensive error handling in agent systems
- Retry logic for transient failures
- Distinguishing between retryable and non-retryable errors
- Graceful degradation and user-friendly error messages

**Production Best Practices:**
- Always validate tool inputs
- Use custom exceptions for clear error handling
- Implement retry logic for network/API failures
- Don't retry on validation errors (waste of resources)
- Provide clear error messages to users
- Log errors for debugging and monitoring

## Capstone Project: AgentHub v1.0

Build a multi-agent platform that routes queries to specialized agents.

### Architecture:

```
User Query → Router Agent → Specialized Agent → Response
                  ↓
            [MathAgent, InfoAgent, TaskAgent]
```

### Step 1: Define Specialized Agents

In [ ]:
class MathAgent:
    """Specialized agent for mathematical operations."""
    
    def __init__(self):
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "calculator",
                    "description": "Performs arithmetic operations",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
                            "a": {"type": "number"},
                            "b": {"type": "number"}
                        },
                        "required": ["operation", "a", "b"]
                    }
                }
            }
        ]
    
    def process(self, query: str) -> str:
        """Process math-related query."""
        messages = [
            {"role": "system", "content": "You are a mathematical assistant. Use the calculator tool to solve problems."},
            {"role": "user", "content": query}
        ]
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=self.tools
        )
        
        response_message = response.choices[0].message
        
        if response_message.tool_calls:
            tool_call = response_message.tool_calls[0]
            args = json.loads(tool_call.function.arguments)
            result = calculator(**args)
            
            messages.append(response_message)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result)
            })
            
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            return final_response.choices[0].message.content
        
        return response_message.content

print("MathAgent defined!")

In [ ]:
class InfoAgent:
    """Specialized agent for information retrieval."""
    
    def __init__(self):
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "search_knowledge_base",
                    "description": "Searches knowledge base",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string"}
                        },
                        "required": ["query"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_current_datetime",
                    "description": "Gets current date and time",
                    "parameters": {"type": "object", "properties": {}}
                }
            }
        ]
    
    def process(self, query: str) -> str:
        """Process information query."""
        messages = [
            {"role": "system", "content": "You are an information assistant. Use tools to find accurate information."},
            {"role": "user", "content": query}
        ]
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=self.tools
        )
        
        response_message = response.choices[0].message
        
        if response_message.tool_calls:
            messages.append(response_message)
            
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                args = json.loads(tool_call.function.arguments)
                
                if function_name == "search_knowledge_base":
                    result = search_knowledge_base(**args)
                elif function_name == "get_current_datetime":
                    result = get_current_datetime()
                else:
                    result = "Unknown tool"
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
            
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            return final_response.choices[0].message.content
        
        return response_message.content

print("InfoAgent defined!")

In [ ]:
class TaskAgent:
    """Specialized agent for task management."""
    
    def __init__(self):
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "create_ticket",
                    "description": "Creates a support ticket",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "title": {"type": "string"},
                            "description": {"type": "string"},
                            "priority": {"type": "string", "enum": ["low", "medium", "high", "urgent"]}
                        },
                        "required": ["title", "description", "priority"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "send_notification",
                    "description": "Sends a notification",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "recipient": {"type": "string"},
                            "message": {"type": "string"},
                            "priority": {"type": "string", "enum": ["low", "normal", "high"]}
                        },
                        "required": ["recipient", "message"]
                    }
                }
            }
        ]
    
    def process(self, query: str) -> str:
        """Process task-related query."""
        messages = [
            {"role": "system", "content": "You are a task management assistant. Help users create tickets and notifications."},
            {"role": "user", "content": query}
        ]
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=self.tools
        )
        
        response_message = response.choices[0].message
        
        if response_message.tool_calls:
            messages.append(response_message)
            
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                args = json.loads(tool_call.function.arguments)
                
                if function_name == "create_ticket":
                    result = create_ticket(**args)
                elif function_name == "send_notification":
                    result = send_notification(**args)
                else:
                    result = "Unknown tool"
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result) if isinstance(result, dict) else str(result)
                })
            
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            return final_response.choices[0].message.content
        
        return response_message.content

print("TaskAgent defined!")

### Step 2: Build AgentHub Router

In [ ]:
class AgentHub:
    """Multi-agent platform with intelligent routing."""
    
    def __init__(self):
        self.agents = {
            "math": MathAgent(),
            "info": InfoAgent(),
            "task": TaskAgent()
        }
    
    def route_query(self, user_message: str) -> str:
        """Determine which agent should handle the query."""
        message_lower = user_message.lower()
        
        # Math keywords
        math_keywords = ["calculate", "compute", "add", "subtract", "multiply", "divide", "math", "sum", "total"]
        if any(keyword in message_lower for keyword in math_keywords):
            return "math"
        
        # Task keywords
        task_keywords = ["ticket", "notify", "notification", "alert", "create", "track", "issue"]
        if any(keyword in message_lower for keyword in task_keywords):
            return "task"
        
        # Default to info agent
        return "info"
    
    def process(self, user_message: str, verbose: bool = True) -> str:
        """Route query to appropriate agent and return response."""
        
        # Route to appropriate agent
        agent_type = self.route_query(user_message)
        
        if verbose:
            print(f"[AgentHub] Routing to {agent_type.upper()} agent")
            print("="*60)
        
        # Get appropriate agent
        agent = self.agents[agent_type]
        
        # Process query
        try:
            response = agent.process(user_message)
            return response
        except Exception as e:
            return f"Error processing request: {str(e)}"

print("AgentHub v1.0 ready!")

### Step 3: Test AgentHub

In [ ]:
# Initialize AgentHub
hub = AgentHub()

# Test 1: Math query
print("Test 1: Mathematical query")
print("="*60)
response = hub.process("Calculate 156 multiplied by 23")
print(f"\nResponse: {response}\n")

# Test 2: Info query
print("\nTest 2: Information query")
print("="*60)
response = hub.process("What are your business hours?")
print(f"\nResponse: {response}\n")

# Test 3: Task query
print("\nTest 3: Task creation query")
print("="*60)
response = hub.process("Create a high priority ticket: Website is loading slowly for users in Europe")
print(f"\nResponse: {response}\n")

# Test 4: Time query
print("\nTest 4: Time information query")
print("="*60)
response = hub.process("What time is it right now?")
print(f"\nResponse: {response}\n")

# Test 5: Complex query
print("\nTest 5: Complex multi-step query")
print("="*60)
response = hub.process("Calculate the sum of 450 and 325, then notify the team lead about the total")
print(f"\nResponse: {response}")

## 🎉 Lab Complete!

Congratulations! You've built a complete multi-agent system.

### What You've Learned:

1. **Tool Calling Fundamentals**
   - Function/tool definitions
   - Tool execution flow
   - OpenAI vs Claude tool calling APIs

2. **Multi-Tool Agents**
   - Defining multiple tools
   - Tool registry pattern
   - Iterative tool calling

3. **Conditional Workflows**
   - Business logic in agents
   - Context-based routing
   - Multi-step workflows

4. **Error Handling**
   - Retry logic
   - Validation vs transient errors
   - Graceful degradation

5. **Multi-Agent Systems**
   - Specialized agents
   - Intelligent routing
   - Agent orchestration

### AgentHub Architecture:

```
AgentHub
├── MathAgent (calculator tools)
├── InfoAgent (knowledge base, datetime)
└── TaskAgent (tickets, notifications)
```

### Production Considerations:

- **Scalability**: Consider async execution for multiple agents
- **Monitoring**: Log agent usage, tool calls, errors
- **Security**: Validate tool inputs, rate limit API calls
- **Cost**: Track token usage per agent
- **Testing**: Unit test each agent and tool separately

## Extension Challenges

Ready to go further? Try these challenges:

### Challenge 1: Add Email Agent
Create a new specialized agent that handles email-related tasks:
- Send email
- Search inbox
- Schedule email
- Update routing logic to include email agent

### Challenge 2: Implement Agent Fallback
If primary agent fails, route to fallback:
- Track agent success/failure rates
- Implement fallback routing logic
- Add metrics and monitoring

### Challenge 3: Add Conversation Memory
Make AgentHub maintain context across messages:
- Store conversation history
- Reference previous queries
- Implement context-aware routing

### Challenge 4: LLM-Based Routing
Replace keyword-based routing with LLM:
- Use LLM to classify query intent
- Support multi-agent queries
- Handle ambiguous queries

### Challenge 5: Add Analytics Dashboard
Track and visualize agent performance:
- Agent usage statistics
- Tool call frequency
- Error rates
- Response times
- Cost per agent

## Key Takeaways

### Tool Calling Patterns:
- Always validate inputs before execution
- Implement retry logic for network operations
- Return structured error messages
- Log all tool calls for debugging

### Agent Design:
- Keep agents specialized and focused
- Use clear system prompts to guide behavior
- Implement proper error handling at every level
- Consider multi-step workflows from the start

### Production Best Practices:
- Monitor agent performance and costs
- Implement rate limiting and quotas
- Add comprehensive logging
- Test edge cases and error scenarios
- Document agent capabilities and limitations

### Next Steps:
- Explore async agent execution
- Implement streaming responses
- Add conversation memory and context
- Build agent analytics and monitoring
- Scale to production workloads